# Deep research of electrocardiograms

In previous tutorials you have learned about batches, pipelines and models. All those concepts are essential components of deep learning research, and CardIO provides convenient implementations of them for all users.
But in order to perform real research, you need to conduct well-described, reproducible experiments and keep track on the results.

For this task `research` module of `batchflow` comes in handy. In this tutorial we will use `research` to train and test multiple variations of ResNet architecture in the task of atrial fibrillation classification.

In this tutorial we will cover part of the functionality of `research`, but we strongly recommend to go through [BatchFlow's Research tutorial](https://github.com/analysiscenter/batchflow/blob/master/examples/tutorials/08_research.ipynb) and to take a look at [documentation for Research](https://analysiscenter.github.io/batchflow/intro/research.html).

# Table of contents



* [Experimental design](#Experimetal-design)
    * [Describing model variations](#Describing-model-variations)
    * [Creating dataset](#Creating-dataset)
    * [Setting up model configuration](#Setting-up-model-configuration)
    * [Setting up training parameters](#Setting-up-training-parameters)
    * [Defining pipelines](#Defining-pipelines)
* [Running the experiment](#Running-the-experiment)
* [Summary](#Summary)

# Experimental design

When you are starting an experiment, the very first thing you should do is to write down the experimental design.  Clear and explicit design of experiment makes it easier to reproduce the results, because it helps to make sure that all important conditions are accounted.

We will now follow this rule and step by step generate experimental desing, which later will be used as configuration for our experiments.

## Describing model variations

First, we shall define which variations of ResNet architecture we are going to test.
`ResNet` class from `batchflow.models` has following configuration parameters:
* number of filters after first convolution
* layout in ResNet blocks
* number of filters in ResNet blocks' convolutions
* number of ResNet blocks between poolings

We are going to tweak all of them! And, we suppose that you are already familiar with models from `batchflow.models`.


Along with custom architectures, let's try some classics, like `ResNet18` and `ResNet34`. To do this, we define "model" option of our experiment:

In [1]:
import sys

sys.path.append("..")

from cardio.batchflow.models.tf import ResNet, ResNet18, ResNet34
from cardio.batchflow.research import Option, Grid, Research

model_op1 = Option('model', [ResNet18, ResNet34])
model_op2 = Option('model', [ResNet])

As you can see, we have defined custom and classic ResNets in different options. Have some patience, in a few moments we will explain this move.

Next, let's define options for input filters:

In [2]:
input_filters_op1 = Option('input_filters', [64])
input_filters_op2 = Option('input_filters', [32, 8])

And now just follow the same procedure for the rest of the parameters:

In [3]:
layout_op1 = Option('layout', ['cnacna'])
layout_op2 = Option('layout', ['cna', 'cnacna'])

blocks_op1= Option('blocks', [[2, 2, 2, 2], [3, 4, 6, 3]])
blocks_op2 = Option('blocks', [[2, 3, 4, 5, 4, 3, 2], [2, 2, 2, 2, 2, 2, 2],
                               [1, 1, 1, 1, 1, 1, 1]])

filters_op1 = Option('filters', [[64, 128, 256, 512], [64, 128, 256, 512]])
filters_op2 = Option('filters', [[4, 8, 16, 32, 64, 128, 256], [4, 4, 8, 8, 16, 16, 20]])

This is just the time to explain why we define two options for each parameter.

We have defined options for different parameters, but what are we going to do with them now? We will use them to generate a single grid of parameters, that will include all the combinations we want to test. 

Here, take a look at this simple example to get the intuition about grid generation:

In [4]:
op1 = Option('p1', [1, 2])
op2 = Option('p2', ['v1', 'v2'])
op3 = Option('p1', [4])
op4 = Option('p2', ['v3'])

grid = (op1 * op2 + op3 * op4)

grid

Grid([[{'p1': ['1', '2']}, {'p2': ['v1', 'v2']}], [{'p1': ['4']}, {'p2': ['v3']}]])

You can perform multipliction and addition of options. But, as far as `Grid` object is not very intuitive, we can generate all configurations from this grid and take a look at them:

In [5]:
list(grid.gen_configs())

[ConfigAlias({'p1': '1', 'p2': 'v1'}),
 ConfigAlias({'p1': '1', 'p2': 'v2'}),
 ConfigAlias({'p1': '2', 'p2': 'v1'}),
 ConfigAlias({'p1': '2', 'p2': 'v2'}),
 ConfigAlias({'p1': '4', 'p2': 'v3'})]

This form is much simpler to understand! 

Also, there is a method `Option.product` that implements element-wise multiplication - see an example below.

In [6]:
grid = Option.product(op1, op2)
    
list(grid.gen_configs())

[ConfigAlias({'p1': '1', 'p2': 'v1'}), ConfigAlias({'p1': '2', 'p2': 'v2'})]

Now we are ready to generate the grid for our experiment:

In [7]:
grid = (Option.product(model_op1, blocks_op1, filters_op1) * layout_op1 * input_filters_op1 + 
        model_op2 * layout_op2 * input_filters_op2 * blocks_op2 * filters_op2)

Great, we're done with model variations. Now we shall move to the next part - the data.

## Creating dataset

Here we just follow familiar procedure: setting up paths to the signals and the labels, then creating dataset instance and splitting data into train and test subsets.

In [8]:
import os
from cardio import batchflow as bf
from cardio import EcgDataset

PATH = "/notebooks/data/ECG/training2017" # Change this path for your data dicrectory
SIGNALS_MASK = os.path.join(PATH, "A*.hea")
LABELS_PATH = os.path.join(PATH, "REFERENCE.csv")

In [9]:
eds = EcgDataset(path=SIGNALS_MASK, no_ext=True, sort=True)
eds.split(0.8, shuffle=False)

## Setting up model configuration

Now, here comes the interesting part! When we define `model_config`, we use `C('parameter_name')` for all the parameters we want to vary:

In [10]:
import tensorflow as tf
from cardio.batchflow import F, B, C, V

model_config = {
    'inputs': dict(signals={'shape': F(lambda batch: batch.signal[0].shape[1:])},
                   labels={'classes': ['A', 'NO'], 'transform': 'ohe', 'name': 'targets'}),
    'initial_block/inputs': 'signals',
    "loss": "ce",
    "input_block/filters": C('input_filters'),
    "body/block/layout": C('layout'),
    "body/filters": C('filters'),
    "body/num_blocks": C('blocks'),
    "session/config": tf.ConfigProto(allow_soft_placement=True),
    "device": C("device"),
    "optimizer": "Adam",
}

Corresponding values from configuration will be inserted instead of those templates.

## Setting up training parameters

This part is pretty short and simple - just setting up batch size and number of epochs.

In [11]:
BATCH_SIZE = 32
EPOCHS = 300

## Defining pipelines

At first we define root pipelines, wich contain only signal processing. Also, we will define a function to flip signals whose R peaks are turned down.

In [12]:
import numpy as np
from numba import njit

@njit(nogil=True)
def center_flip(signal):
    """
    Center signal and flip signals with R peaks turned down.
    """
    return np.random.choice(np.array([1, -1])) * (signal - np.mean(signal))

In [13]:
root_train = (
  bf.Pipeline()
    .load(components=["signal", "meta"], fmt="wfdb")
    .load(components="target", fmt="csv", src=LABELS_PATH)
    .drop_labels(["~"])
    .rename_labels({"N": "NO", "O": "NO"})
    .apply_to_each_channel(center_flip)
    .random_resample_signals("normal", loc=300, scale=10)
    .random_split_signals(3000, {"A": 6, "NO": 2})
    .apply_transform(func=np.transpose, src='signal', dst='signal', axes=[0, 2, 1])
).run(BATCH_SIZE, shuffle=True, drop_last=True, n_epochs=None, lazy=True)

root_test = (
  bf.Pipeline()
    .load(components=["signal", "meta"], fmt="wfdb")
    .load(components="target", fmt="csv", src=LABELS_PATH)
    .drop_labels(["~"])
    .rename_labels({"N": "NO", "O": "NO"})
    .apply_to_each_channel(center_flip)
    .split_signals(3000, 3000)
    .apply_transform(func=np.transpose, src='signal', dst='signal', axes=[0, 2, 1])
).run(BATCH_SIZE, shuffle=True, drop_last=True, n_epochs=1, lazy=True)


Next, we define main pipelines. Training pipeline is pretty simlpe - we just train the model and save loss to pipeline variable.
Again, we need to define simple function - this one prepares data from batch to be fed into a model.

In [14]:
def make_data(batch, **kwagrs):
    """
    Prepare data from `signal` and `target` component of batch to be fed into network.
    Separates each crop as individual signal and makes a corresponding label.
    """
    n_reps = [signal.shape[0] for signal in batch.signal]
    signals = np.array([segment for signal in batch.signal for segment in signal])
    targets = np.repeat(batch.target, n_reps, axis=0)
    return {"feed_dict": {'signals': signals, 'labels': targets}}

In [15]:
model_train = (
  bf.Pipeline()
    .init_variable('loss', init_on_each_run=list)
    .init_model('dynamic', C('model'), 'model', config=model_config)
    .train_model('model',
                 make_data=make_data,
                 fetches=["loss"],
                 save_to=[V("loss")], mode="w")
)

Testing pipeline is a bit more complex. At first, we initialize a variable to store metrics and save number of splits for each signal in batch component `splits`. Then, as usual, import model, make predictions and save predictions and targets into the batch components.

As far as we make predictions for splits, we need to aggrageate them to obtain prediction for the whole signal. To do so, we will use `aggregate_crop_predictions` and update corresponding batch components.
Then, pipeline method `gather_metrics` accumulates targets and predictions from batch and allows to calculate metrics afterwards.

In [16]:
def aggregate_crops(arr, splits, agg_func, predictions=True, **kwargs):
    """
    Aggregates predictions or labels from separate crops for whole signal.
    If predictions is True, applies softmax function before aggreagation.
    """
    if predictions:
        arr -= np.max(arr, axis=1, keepdims=True)
        arr_exp = np.exp(arr)
        arr = (arr_exp / np.sum(arr_exp, axis=1, keepdims=True))
    arr = np.split(arr, np.cumsum(splits)[:-1])
    return np.array([agg_func(sig[:, 0]) for sig in arr])

In [17]:
model_test = (
  bf.Pipeline()
    .init_variable("metrics", init_on_each_run=None)
    .apply_transform(src="signal", dst="splits", func=lambda x: [x.shape[0]])
    .import_model("model", C("import_from"))
    .predict_model("model", make_data=make_data,
                   fetches=["predictions", "targets"], 
                   save_to=[B('predictions'), B("targets")])
    .apply_transform_all(func=aggregate_crops, predictions=True,
                         src='predictions', dst='predictions', 
                         splits=B('splits'), agg_func=np.mean)
    .apply_transform_all(func=aggregate_crops, predictions=False, 
                         src='targets', dst='targets',
                         splits=B('splits'), agg_func=np.max)
    .gather_metrics('class', targets=B('targets'), predictions=B('predictions'),
                    fmt='proba', save_to=V('metrics'), mode='a')
)

# Running the experiment

Before starting the experiment, we have to set how often to run test pipeline. Calculation of this value is not very strainghtforward because it should be stated in a number of iterations, not epochs.

In [18]:
TEST_EACH_EPOCH = 20
TRAIN_SIZE = len(eds.train)
ITERATIONS = ((TRAIN_SIZE // BATCH_SIZE) + 1) * EPOCHS
TEST_EXEC_FOR = ITERATIONS // EPOCHS * TEST_EACH_EPOCH
STR_EXEC = '%{}'.format(TEST_EXEC_FOR)

Also, we will set a few constants for research:

In [19]:
N_REPS = 5
N_BRANCHES = 2
N_WORKERS = 3
GPU = [1, 2, 3, 4, 5, 6]

At the end of run of any pipeline we can execute some function. So let's write two functions to save number of trainable parameters and F1 score: 

In [20]:
def get_trainable_variables(iteration, experiment, ppl, model_name="model"):
    """
    Returns a number of trainable variables in the model.
    """
    return experiment[ppl].pipeline.get_model_by_name(model_name).get_number_of_trainable_vars()


def calc_metrics(iteration, experiment, ppl, var_name):
    """
    Gets variable with metrics class from the pipeline and
    calculates F1 score.
    """
    metrics = experiment[ppl].pipeline.get_variable(var_name)
    f1 = metrics.evaluate('f1_score')
    return f1

Now, define the whole research pipeline and run it:

In [21]:
mr = (
    Research()
    .pipeline(root_train << eds, model_train,
              variables=["loss"], name="train", dump=STR_EXEC)
    .pipeline(root_test << eds, model_test,
              name="test", execute=STR_EXEC, dump=STR_EXEC,
              import_from="train", run=True)
    .function(calc_metrics, returns='metrics_f1', name='metrics_f1',
              execute=STR_EXEC, dump=STR_EXEC, ppl='test', var_name='metrics')
    .function(get_trainable_variables, returns='trainable_variables', 
              name='trainable_variables', execute=1, dump=1, ppl='train')
    .grid(grid)
)

In [22]:
mr.run(n_reps=N_REPS, n_iters=ITERATIONS, workers=N_WORKERS, gpu=GPU, 
       branches=N_BRANCHES, name='ResNet_research', progress_bar=True)

Now in folder ResNet_research you can find the results: loss, saved after each iteration, f1 score on the whole test set calculated each 20 epochs and number of trainable variables in each model.

# Summary

In this notebook we have learned about research and experiments and now you know how to:
* define research grid
* use metrics in the pipeline
* run research experiments